In [4]:
import cohere
import tkinter as tk
from tkinter import Canvas, Frame, Scrollbar, BooleanVar
from datetime import datetime
import platform
import pyttsx3

# --- CONFIGURATION ---
API_KEY = "your Cohere API key"  # 🔐 Replace with your Cohere API key
MODEL = "command-r"

co = cohere.Client(API_KEY)
chat_history = []

# --- GUI Setup ---
root = tk.Tk()
root.title("AI Chatbot – WhatsApp Style")
root.geometry("400x640")
root.configure(bg="#ECE5DD")
root.resizable(False, False)

voice_reply_enabled = BooleanVar(value=False)  # ✅ Moved here after root is created

# --- TTS Engine Setup ---
tts_engine = pyttsx3.init()
tts_engine.setProperty("rate", 165)  # Voice speed

def speak(text):
    tts_engine.say(text)
    tts_engine.runAndWait()

def get_bot_response(user_input):
    global chat_history
    try:
        response = co.chat(
            model=MODEL,
            message=user_input,
            chat_history=chat_history
        )
        chat_history = response.chat_history
        return response.text.strip()
    except Exception as e:
        return f"[Error] {str(e)}"

def get_time():
    return datetime.now().strftime("%I:%M %p")

def add_message(text, sender="user"):
    is_user = sender == "user"
    bubble_color = "#DCF8C6" if is_user else "#FFFFFF"
    align = "e" if is_user else "w"
    name = "You" if is_user else "AI"

    msg_frame = Frame(chat_frame, bg="#ECE5DD")
    header = tk.Label(msg_frame, text=f"{name} • {get_time()}", font=("Helvetica", 8), bg="#ECE5DD", fg="gray")
    header.pack(anchor=align, padx=5)

    bubble = Frame(msg_frame, bg=bubble_color, padx=10, pady=7)
    msg = tk.Label(bubble, text=text, bg=bubble_color, wraplength=280, justify="left", font=("Helvetica", 12))
    msg.pack()
    bubble.pack(anchor=align, padx=10, pady=2, ipadx=4, ipady=2)

    msg_frame.pack(anchor=align, fill="x")
    chat_canvas.update_idletasks()
    chat_canvas.yview_moveto(1.0)

def send_message():
    user_input = entry.get().strip()
    if not user_input:
        return
    entry.delete(0, tk.END)
    add_message(user_input, sender="user")
    root.after(100, lambda: get_and_show_bot_response(user_input))

def get_and_show_bot_response(user_input):
    bot_response = get_bot_response(user_input)
    add_message(bot_response, sender="bot")
    if voice_reply_enabled.get():
        speak(bot_response)

# --- Chat Canvas Setup ---
chat_canvas = Canvas(root, bg="#ECE5DD", highlightthickness=0)
chat_scrollbar = Scrollbar(root, orient="vertical", command=chat_canvas.yview)
chat_canvas.configure(yscrollcommand=chat_scrollbar.set)

chat_scrollbar.pack(side="right", fill="y")
chat_canvas.pack(side="top", fill="both", expand=True)

chat_frame = Frame(chat_canvas, bg="#ECE5DD")
chat_window = chat_canvas.create_window((0, 0), window=chat_frame, anchor='nw')

def on_frame_configure(event):
    chat_canvas.configure(scrollregion=chat_canvas.bbox("all"))

chat_frame.bind("<Configure>", on_frame_configure)

def _on_mousewheel(event):
    if platform.system() == 'Windows':
        chat_canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")
    elif platform.system() == 'Darwin':
        chat_canvas.yview_scroll(int(-1 * event.delta), "units")
    else:
        chat_canvas.yview_scroll(int(-1 * (event.delta)), "units")

chat_canvas.bind_all("<MouseWheel>", _on_mousewheel)
chat_canvas.bind_all("<Button-4>", lambda e: chat_canvas.yview_scroll(-1, "units"))
chat_canvas.bind_all("<Button-5>", lambda e: chat_canvas.yview_scroll(1, "units"))

# --- Bottom Controls ---
bottom_frame = Frame(root, bg="#ECE5DD")
bottom_frame.pack(fill='x', pady=5)

entry = tk.Entry(bottom_frame, font=("Helvetica", 12), bg="white")
entry.pack(side="left", padx=(10, 5), pady=5, fill='x', expand=True)
entry.bind("<Return>", lambda event: send_message())

send_btn = tk.Button(bottom_frame, text="Send", bg="#25D366", fg="white", font=("Helvetica", 12, "bold"), command=send_message)
send_btn.pack(side="right", padx=(0, 5), pady=5)

# --- Voice/Text Toggle ---
toggle_frame = Frame(root, bg="#ECE5DD")
toggle_frame.pack(fill='x', padx=10)

voice_check = tk.Checkbutton(toggle_frame, text="🔈 Voice Reply", variable=voice_reply_enabled, bg="#ECE5DD", font=("Helvetica", 10))
voice_check.pack(anchor="w")

# --- Start App ---
root.mainloop()
